# Model for Simulating Energy and Economic Performance of Off-Grid PV in DR Aggregations V1.0

In [1]:
import pandas as pd
import requests
import pvlib
import matplotlib.pyplot as plt  # for visualization
import math
from datetime import datetime
from statistics import median
from statistics import mean
from keys import *

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
#GLOBALS
#note that these coordinates are the inputs for the NREL API,
#the coordinates used by the NREL API for the weather file are slightly different
#see below
lat = 40.7128
long = -74.0060

#optimize for summer when DR program is in effect
#also minimizes shading from buildings during PSH
tilt = math.floor(lat - 15)
print("tilt: " + str(tilt))

#default to south facing
azimuth = 180

#14% losses
shading = .86

#max 99% efficient mppt battery charger
mppt = .99

#max 80% depth of discharge for LiFePO4 batteries
LiFePOdod = .8

#Max Hardware Sizes
#800W is the max in Germany for balcony solar
pvMaxW = 800
#2kWh is the largest system FDNY allows without additional beurocracy 
#batMaxWh = 2000

#PV degredation rate is assumed to be 1% annually
DEG_RATE = 0.01



## Commercially Available Power Station Data


In [ ]:
hardware_df = pd.read_csv('data/PowerStations_LiFePo4_Jan31_2025.csv')

#clean data - these columns aren't used in this analysis
hardware_df = hardware_df.drop(columns=['Link','Min PV Voltage','Max PV Voltage','Warranty Years','Lifespan','Notes','Estimated Full Charge Time','Estimated Minimum Grid Charge Efficiency'])

retailPriceColumns = ['Retail Price (1/24/2025)','Retail Price (1/30/2025)']

#filter out batteries over specified size
#print(hardware_df[hardware_df['Battery Wh'] > 2000])
#hardware_df = hardware_df[hardware_df['Battery Wh'] <= 2000]

hardware_df.head()

In [ ]:
pC = []
for c in hardware_df.columns:
    if 'Price (' in c:
        pC.append(c)

#get average retail prices
hardware_df['retail price']= hardware_df[retailPriceColumns].mean(axis=1)
hardware_df = hardware_df.drop(columns=pC) #drop raw price data

hardware_df = hardware_df.dropna(subset=['retail price']) #drop rows with missing avg prices

hardware_df.head()

In [ ]:
#display(hardware_df.tail())

#get average charge time percent for 80% relative to a 100% charge
hardware_df['charge ratio']= hardware_df['80 Per Grid Charge Hours']/hardware_df['Full Grid Charge Hours']

#display(hardware_df[~hardware_df['charge ratio'].isnull()]['charge ratio'])
#get the average ratio of 80% to 100% 
eightyCT = round(hardware_df[~hardware_df['charge ratio'].isnull()]['charge ratio'].mean()*100,2)*.01
print(str(eightyCT*100) + '% of the full charge time is spent charging from 0-80%.')
# #display(hardware_df.tail())

#update full charge column
hardware_df['Full Grid Charge Hours'].loc[hardware_df['Full Grid Charge Hours'].isnull()] = hardware_df['80 Per Grid Charge Hours'].loc[hardware_df['Full Grid Charge Hours'].isnull()] / eightyCT

#update 80% charge column
hardware_df['80 Per Grid Charge Hours'].loc[hardware_df['80 Per Grid Charge Hours'].isnull()] = hardware_df['Full Grid Charge Hours'].loc[hardware_df['80 Per Grid Charge Hours'].isnull()] * eightyCT

In [ ]:
tP = hardware_df.shape[0]

inverterEfficiencyMean = round(hardware_df['Inverter Efficiency'].mean(),4)
invEffDefault = inverterEfficiencyMean * 0.01
print('Default Efficiency for use in model: ' + str(inverterEfficiencyMean))
print('% with unknown efficiency: ' + str(100*hardware_df[hardware_df['Inverter Efficiency'].isnull()].shape[0]/tP))

hardware_df['Inverter Efficiency'] = hardware_df['Inverter Efficiency'].fillna(inverterEfficiencyMean)
hardware_df['Inverter Efficiency'] = hardware_df['Inverter Efficiency']*.01
display(hardware_df.tail())

In [ ]:
hardware_df.info()

In [ ]:
hardware_df.describe()

In [ ]:
# data snapshot
print('Total Number of Companies: ' + str(hardware_df.drop_duplicates(subset=['Make']).shape[0]))
print('Total Number of Products: ' + str(tP))

print('')
print('*** PV Charge Time (without losses) ***')
print('Min:' + str(min(hardware_df['Battery Wh']/(hardware_df['Max PV Watts']*.99))))
print('Max:' + str(max(hardware_df['Battery Wh']/(hardware_df['Max PV Watts']*.99))))
pvChargeMed = (hardware_df['Battery Wh']/(hardware_df['Max PV Watts']*.99)).median()
print('Median:' + str(pvChargeMed))


## Con Ed Network Data

In [ ]:
# read in hardware data to dataframe
networks_df = pd.read_csv('data/conEdNetworks2024.csv')

networks_df.head()

In [ ]:
#returns the network info for a given network
#args: network name
def getNetworkInfo(networkName):
    return networks_df.loc[networks_df['network']== networkName]

#returns the tier for a given item in the network info list
#args: network name
def getDLRPrate(networkName):
    n = getNetworkInfo(networkName)
    t = n['tier'].iloc[0]
    #tier 2 networks pay at a higher rate
    if t == 1:
        return 18
    else:
        return 25
    
#returns the borough for a given item in the network info list
#args: network name
def getCSRPrate(networkName):
    n = getNetworkInfo(networkName)
    t = n['borough'].iloc[0]
    #westchester and staten island pay at a lower rate
    if t not in ['westchester','staten island']:
        return 18
    else:
        return 6

#args: network name
def getTotalRate(networkName):
    return getDLRPrate(networkName) + getCSRPrate(networkName)
    
#returns borough name
#args: network name
def getBorough(networkName):
    n = getNetworkInfo(networkName)
    return n['borough'].iloc[0]

#returns event start time
#args: network name
def getStartTime(networkName):
    n = getNetworkInfo(networkName)
    return int(n['start time'].iloc[0].split(":")[0])

## PV Performance Modeling

In [ ]:
#get TMY via API
df_tmy, metadata = pvlib.iotools.get_psm3(
    latitude=lat, longitude=long,
    api_key=NREL_API_KEY,
    email='nathaa@rpi.edu',  # <-- any email works here fine
    names='tmy-2020')
metadata

In [ ]:
# make a Location object corresponding to this TMY
location = pvlib.location.Location(name='NYC',
                                   latitude=metadata['latitude'],
                                   longitude=metadata['longitude'])

location

In [ ]:
#normalize all years to 2020
#remove timezone info #,tzinfo=None
df_tmy.index = df_tmy.index.map(lambda t: t.replace(year=2020))

#uncomment the 30 minute shifts if using data that is right-labeled at hourly intervals!
# Note: TMY datasets are right-labeled hourly intervals, e.g. the
# 10AM to 11AM interval is labeled 11.  We should calculate solar position in
# the middle of the interval (10:30), so we subtract 30 minutes:
times = df_tmy.index #- pd.Timedelta('30min')
solar_position = location.get_solarposition(times)
# but remember to shift the index back to line up with the TMY data:
#solar_position.index += pd.Timedelta('30min')

solar_position

In [ ]:
#get monthly temperature averages for validation
# for m in range(12):
#     df_tmy['Temperature']

In [ ]:
gamma_pdc = -0.004  # divide by 100 to go from %/°C to 1/°C

all_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']
parameters = all_parameters['open_rack_glass_polymer']

#args: tilt, azimuth
def getPOA(t,a):
    #fixed tilt POA
    df_poa = pvlib.irradiance.get_total_irradiance(
        surface_tilt=t,
        surface_azimuth=a,
        dni=df_tmy['dni'],
        ghi=df_tmy['ghi'],
        dhi=df_tmy['dhi'],
        solar_zenith=solar_position['apparent_zenith'],
        solar_azimuth=solar_position['azimuth'],
        model='isotropic')

    #print(df_poa.keys())
    return df_poa

#DC power going in to battery after accounting for shading and equipment derating
#https://pvlib-python.readthedocs.io/en/stable/reference/generated/pvlib.pvsystem.pvwatts_dc.html#
#args: power, temp coefficient, array nameplate watts
def getPowerYield(p,g,n):
    
    # note the "splat" operator "**" which expands the dictionary "parameters"
    # into a comma separated list of keyword arguments
    cell_temperature = pvlib.temperature.sapm_cell(
        p['poa_global'], df_tmy['temp_air'], df_tmy['wind_speed'], **parameters)
    
    array_power = pd.DataFrame({
        'power': pvlib.pvsystem.pvwatts_dc(p['poa_global'], cell_temperature, n, g,temp_ref=25.0)
    })

    #yield is actual power going in to battery after losses
    array_power['yield'] = array_power['power'] * shading * mppt
    
    return array_power
    #array_power = pvlib.pvsystem.pvwatts_dc(df_poa['poa_global'], cell_temperature, n, g,temp_ref=25.0)

    #factor in 14% shading losses + 1% mppt losses
    #return array_power * shading * mppt

#convert DC to AC with PV Watts model
#https://pvlib-python.readthedocs.io/en/stable/reference/generated/pvlib.inverter.pvwatts.html#pvlib.inverter.pvwatts
#args: dc power input to inverter, inverter nameplate max WAC output, nameplate efficiency
def getACoutput(dcP,maxACW,iE):
    dcPMax = maxACW/iE
    return pvlib.inverter.pvwatts(dcP, dcPMax,iE)

#returns the degraded power of the array for a given year
#args: PV watts, year, PV degradation rate
def getDegradedP(w,y,dR = DEG_RATE):
    d = 1 - dR
    return w*(d**y)

array_power_yield = getPowerYield(getPOA(tilt,azimuth),gamma_pdc, 100)
array_power_yield.head()

In [ ]:
# print(array_power_yield['power'].max())
# print(array_power_yield['yield'].max())
# #array_power_yield.iloc[2891]

In [ ]:
array_power_yield.head(24*7).plot()
plt.ylabel('Array Power [WDC]');

## Economic Modeling & Hardware Costs

Components:
* CAPEX
* Annual Economic Benefit
* grid electricity costs
* NPV
* LCOSS

In [ ]:
#average inflation rate for 2023, based on NREL data from https://atb.nrel.gov/electricity/2023/definitions#inflation
aIR = 0.0253

discountRate = 0.03 #3% discount rate (5.5 is used in some NREL docs)

#returns inflation rate after Y years
#args: year
def infRate(y, r= aIR):
    return (1+r)**y

#argues value to be inflated, year
def inflateIt(v,y):
    return v* infRate(y)
    
#args: value to be discounted, year
def discountIt(v, y,r =discountRate):
    return v/(1+r)**y

inflateIt(10,4)

### Capital Expenses (CAPEX) i.e. upfront costs

In [ ]:
# Upfront Costs

#args: pv watts
def getPVcost(w):
    #.7 is based on a 30% tax credit on PV modules
    taxCredit = .3
    #assumes $1 per W modules
    modulePerW = 1.0

    return w * (1-taxCredit) * modulePerW

#this is the total initial expenditure
#args: pv watts, battery price
def getCAPEX(pvW,batPr):
    #total sales tax in NYC is 8.875%
    salesTax =  0.08875 
    
    #variable costs
    mountDollarPerW = .4

    #fixed costs
    insulation = 4.47 #used for insulating cable entry
    smartController = 115.58
    operatingCosts = 0 #DR aggregation operator costs are not considered here.
    pvCables = 17.99 #2x 10ft MC4 cables
    fixedHardwareCosts = smartController + operatingCosts + insulation + pvCables

    variableCosts = batPr + getPVcost(pvW) + (pvW * mountDollarPerW)

    return (fixedHardwareCosts + variableCosts) * (1 + salesTax)

#print(getCAPEX(100,200))


## Annual Economic Benefit

### Grid Electricity Costs

In [ ]:
# GRID ELECTRICITY COSTS

# https://www.bls.gov/regions/northeast/news-release/averageenergyprices_newyork.htm accessed January 24, 2025
monthlyResElecPricesDollars = [0.265,0.255,0.266,0.259,0.252,0.288,0.288,0.276,0.273,0.274,0.262,0.272]

avgDollarsKw =round(sum(monthlyResElecPricesDollars)/len(monthlyResElecPricesDollars),4) 
print('12 Month NYS Residential Retail Electricity Average: $' + str(avgDollarsKw) + '/kWh')

#args: kWAC generated from PV, year from installation
def utilityCosts(gkWAC,y=0):
    
    #utility costs, with grid rate increasing with inflation
    annualElectCosts = gkWAC * inflateIt(avgDollarsKw,y)
    
    return annualElectCosts

print(utilityCosts(15,3))

### Net Present Value (NPV)

In [ ]:

# args:
# working life, capex, annual avoided grid costs,
# monthly DR revenue C, monthly DR revenue D,
# annual DR performance income, annual electricity costs
# all of these inputs should be adusted for inflation
def getNPV(wY,c, aGC, drInc_C, drInc_D, drInc_A,gC):    
    tAB = 0 #annual benefit = avoided electricity cost + DR income
    tOM = 0 #O&M = electricity expense
    
    #get full years
    yV = 0 #value by 
    for y in range(math.floor(wY)):
        tOM = gC[y]
        tAB = aGC[y] + drInc_A[y]
        num = tAB - tOM
        # print('num: ' + str(num))
        # print('year: ' + str(y))
        # print('discounted num: ' + str(discountIt(num,y,discountRate)))
        yV += discountIt(num,y,discountRate) 

    #remaining months
    rY = math.floor(wY)
    
    if wY < 10:
        tOM = gC[rY] * (wY - rY) #grid costs for the remaining portion of the year
        tAB = aGC[rY] * (wY - rY) #avoided grid costs
        
        # these are the months during the DR season
        remainingMonths = min(5,math.ceil((wY - math.floor(wY))*12)  - 5)
        if remainingMonths > 0:
            for m in range(0,remainingMonths):
                tAB += drInc_C[rY][m] + drInc_D[rY][m]
                
        num = tAB - tOM
        
    yV += discountIt(num,rY,discountRate)
        
    n = yV - c
    return n

#single vals: working life, capex
#lists: annual avoided grid costs, annual DR performance income,  annual electricity costs
# list of list: monthly DR revenue C, monthly DR revenue D,
#getNPV(8.58904109589041,1066.3, [46.906537336134555,47.37939249657249,47.85346913668602,48.32863652871013,48.80475820540271,49.28169177824046,49.759288750464584,50.23739432483748, 50.71584720596855, 51.19447939706274],[[5.376009646560273, 5.384158899275259, 5.361037986916802, 5.2798520136706175, 5.134510669910331], [5.3543540853524885, 5.362421918214469, 5.339546885805801, 5.2591882795345795, 5.115265161090421], [5.332914540770085, 5.340901766531376, 5.318270181660139, 5.238730678436593, 5.096211666625845], [5.311688863372641, 5.3195962866868145, 5.2972057407396775, 5.218477159081913, 5.077348275143635], [5.2906749250007, 5.298503342502998, 5.276351450438105, 5.198425690489033, 5.058673094210016], [5.269870618567202, 5.277620818954158, 5.25570521907566, 5.178574261790526, 5.040184250144493], [5.249273857850922, 5.256946621959205, 5.235264975693834, 5.158920882035803, 5.021879887835713], [5.228882577291896, 5.2364786781763595, 5.215028669852045, 5.139463579995751, 5.003758170559118], [5.208694731788806, 5.216214934799785, 5.194994271426283, 5.120200403969279, 4.985817279796348], [5.18870829649831, 5.196153359358144, 5.175159770409666, 5.101129421591681, 4.9680554150563765]], [[3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899], [3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899], [3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899], [3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899], [3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899], [3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899], [3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899], [3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899], [3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899], [3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899, 3.406105535539899]], [3.5380758955111045, 3.524103510666368, 3.5102705112032044, 3.4965755100032907, 3.483017133685447, 3.469594022470938, 3.4563048300500636, 3.443148223450023, 3.430122882904067, 3.4172275017218903], [0.9243162947368424, 0.9477014969936846, 0.9716783448676249,0.996261806992776, 1.0214672307096933, 1.0473103516466484, 1.0738073035433089,1.1009746283229547,1.1288292864195255, 1.1573886673659397]) 


### Levelized Cost of Solar plus Storage (LCOSS)

In [ ]:
#returns the dollar value of the module at the end of the system's working like with inflation
# factors in module degradation 
# args: PV watts nameplate,year
def getResidualValue(w,y):
     return inflateIt(getPVcost(getDegradedP(w,y)),y)

#test
#getResidualValue(100, 8.3)

In [ ]:
'''
Numerator = CAPEX + Depreciation Benefits + O&M + Cost of purchased electricity - RESIDUAL VALUE
Denomenator = ANNUAL PV WAC Electricity Yield to Grid + ELEC FROM GRID TO BATTERY AND BACK - SELF CONSUMPTION
'''

# LEVELIZED COST OF SOLAR plus STORAGE - VERY OUT OF DATE - NEEDS COMPLETE OVERHAUL IF BEING USED
#pv nameplat watts, battery nameplate watt hours, capex, working years, annual PV yield kWAC (list), effective efficiency of inverter, opex, avg annual events
def getLCOSS(w, wH, cap,wY,aY,eE,oM=0,aE = 4):
    #grid energy costs are based on the assumption that the battery is charged
    #from 20% to 100% before every DR event and that there are 14 events per year
    #this only considers CSRP events which average 3 per year, not DLRP events which average less than 1 a year
    annEv = aE #14 annual events
    bC = wH * LiFePOdod * annEv #annual battery capacity is the nameplate Wh * 80% depth of discharge * the number of events
    aW = bC * eE #Available Watts AC, assumes a 4 hour discharge rate
    pW = bC / .95 #assumes 95% efficient AC battery charging
    
    num = cap  #intialize numerator with CAPEX
    den = 0 #kW
    
    #get all sums
    for mY in range(math.floor(wY)):
        if oM > 0:
            num += discountIt(oM,mY,discountRate) #O&M
        num += discountIt(utilityCosts(pW*.001,mY),mY,discountRate) #UTILITY COSTS are in kW
        den += discountIt(pW,mY,discountRate) #PV kWAC degraded #confirm that this gets discounted
        den += discountIt(aY[mY],mY,discountRate) #Stored Energy from the grid kWAC #confirm that this gest discounted
    
    #remaining months
    if wY < 10:
        rY = math.floor(wY)
        #round up to a full month
        remainingMonths = math.ceil((wY - math.floor(wY))*12)

        #this does NOT consider uneven power and capital flows over the course of the year
        for mY in range(remainingMonths):
            num += discountIt(oM,mY,discountRate)/12 #O&M
            num += discountIt(utilityCosts(pW *0.001,mY),mY,discountRate)/12 #UTILITY COSTS...is pW in kW???
            den += discountIt(pW *0.001,mY,discountRate)/12 #PV kWAC degraded
            den += discountIt(aY[rY],mY,discountRate)/12 #Stored Energy from the grid kWAC #confirm that this gest discounted

  #subtract the value of the PV module at the end of the battery's life
    num -= getResidualValue(w,wY)
   
    return num/den

# #test
# ayT = [56.05835348289077, 55.497769948061865, 54.94279224858124, 54.39336432609544, 53.84943068283447,
#        53.31093637600613, 52.77782701224607,52.25004874212361, 51.72754825470238, 51.21027277215535]
# print("LCOSS: $" + str(getLCOSS(300, 500,1000,8.5, ayT,.9)) + "/per kWAC")

## Max Automated DR Participation

In [ ]:
# self consumption of both controller and power station
controllerSC = 2.5 #controller self-consumption
powerStationSC = 10.2 #power station no-load self-consumption with AC and DC outputs turned on
scW = controllerSC + powerStationSC
dailySelfConsumptionWh = scW * 24
annualSelfConsumptionkWh = scW * 24 * 365 * 0.001

### CSRP

In [ ]:
#Max automated battery + PV generation during event windows from May through September
#assumes battery is fully charged (from either PV or grid) at start of event
#returns average hourly commitment i.e. reservation amount in watts AC
# i.e. it returns what can automatically be provided per hour for 4 hours during a event in a given month

#get the available hourly watts if the battery is fully depleted over 4 hours
#arg: battery Wh
def getFourHourW(bWh):
    return bWh * LiFePOdod * .25

#returns a list with the average automated participation for each of the 5 months
#args: battery Wh, hourly PV W yield, inverter efficiency, start time, inverter max watts, self-consumption=0, year=0
def performanceCSRP(batWh, pvWyield,iE,sT,iM,sC = 0, year=0):
    #divide available Wh by 4 to account for the 4-hour long event window
    #this assumes a full battery at the onset of the event
    #subtract self-consumption
    batteryACOutput = getACoutput(getFourHourW(batWh)-sC,iM,iE)
    
    runningTot = 0

    #days in the months May - September
    daysInMonth = [31,30,31,31,30]
    
    #returns average WAC generated for all window hours on a monthly basis
    #Get all the DR event window hours from May 1st (day #121) through September 30th (day # 273)
    elapsedDays = 121
    drEventWhAC_MonthlyAvg = []
    #loop through the months
    for m in range(len(daysInMonth)):
        #print(daysInMonth[m])
        drEventWhAC_DailyAvg = 0
        #loop through days in the month
        for d in range(elapsedDays, elapsedDays +  daysInMonth[m]):
            #print(d)
            drEventWhAC = 0
            #loop through the 4 hours during the event
            for e in range(sT,sT + 4):
                #PV Yield for 1 hour during the event window
                drHour = ((d-1)*24)+e
                #print(getACoutput(pvWyield[drHour],iM,iE))
                drEventWhAC += getACoutput(getDegradedP(pvWyield[drHour],year),iM,iE)
            #print(drEventWhAC)
            #get the average for the day
            drEventWhAC_DailyAvg += drEventWhAC * .25
        #print((drEventWhAC_DailyAvg/daysInMonth[m]) + batteryACOutput)
        drEventWhAC_MonthlyAvg.append((drEventWhAC_DailyAvg/daysInMonth[m]) + batteryACOutput)
        drEventWhAC_DailyAvg = 0
        elapsedDays += daysInMonth[m]
    
        #this is averaged to an hourly basis NOT the total production during the event
        #runningTot = runningTot + (drEventDCWh/4)
    
    numDays = 274-121
    #print(drEventWhAC_MonthlyAvg)
    return drEventWhAC_MonthlyAvg

#uncomment to test
performanceCSRP(2000, array_power_yield['yield'], invEffDefault,11,500,5)

### DLRP

In [ ]:
#morning yeild for DLRP events
#this used for determining starting battery capacity, so should be returned DC with mppt losses
#returns average PV Wh yield DC that can be stored in the battery prior to a given time of the day in July
#use 2pm for contingency event starts and 10am for immediate event starts
#args PV yield, event type, year for PV degradation
def morningYield(pvWyield, event='c',year=0):
    #contingency event
    t = 18
    if event == 'i':
        #immediate event
        t = 14

    #loop through 31 days in July
    days = 31
    startDay = 182
    dailyAvg = 0
    for d in range(startDay, startDay + days):
        #loop through hours in the day
        dailyPV = 0
        for e in range(0,t):
            #PV Yield for 1 hour during the event window
            myHour = ((d-1)*24)+e
            #print(getACoutput(pvWyield[drHour],iM,iE))
            dailyPV += getDegradedP(pvWyield[myHour],year)*.99
        dailyAvg += dailyPV
    
    return dailyAvg / days

morningYield(array_power_yield['yield'],'i')

In [ ]:
#returns the DC Wh the battery charged up to in 2 hours of charging from the grid
#does not include the bottom 20%
#arga: battery capacity, full grid charge time, top 20% charge time
def twoHourChargeWh(batWh,gridPT, gridT):
    maxB = batWh * .8
    #time it takes to charge from 20% to 80% percent
    e = gridPT - gridT #* eightyCT * .75
    #print(e)
    #time it takes o
    t = gridPT
    #print(t)
    #print(e+t)
    #if 80% is reached in under 2 hours
    if e < 2:
        #print('80 under 2')
        #remaining hours
        return min(maxB,(batWh * .2 * ((2-e)/t)) + (batWh * .6))
    else:
        #print('80 not under 2')
        #if 20%-80% isn't reached in 2 hours
        #prorate that chunk 
        return batWh * .6 * (2/e)
    
    #chargeAmt = min(1,2/(gridT*.8))

#returns a list with the maximum automated participation for each of the 5 months
#args: battery Wh, grid charge hours 20-100%, grid charge time 80-100%, hourly PV W yield, inverter efficiency, inverter max watts, self-consumption=0, year=0, event
def performanceDLRP(batWh, gridPT, gridT, pvWyield,iE,iM,sC = 0, year=0,event='c'):
    
    maxWh = getFourHourW(batWh)
    
    #get the available PV yield from the morning spread out over 4 hours of event time
    mY = morningYield(pvWyield, event,year)*.25
    
    #this assumes battery wasn't depleted below 20%
    if event == 'c':
        #DLRP assumes charged as much as possible in 2 hours. It doesn't assume a full battery.
        
        #if grid charge time from 20%-100% > 2 hours
        if gridPT > 2:
            availableWh = twoHourChargeWh(batWh,gridPT, gridT) * 0.25
        else:
            availableWh = getFourHourW(batWh)        
        
        availableWh = min(maxWh, availableWh + mY)
    else:
        #for immediate events with no time to charge the battery from the grid,
        #the available energy is based off of PV production from earlier in the day
        availableWh = min(maxWh, mY)
        
    #subtract self-consumption and account for inverter losses
    batteryACOutput = getACoutput(availableWh-sC,iM,iE)

    drEventWhAC_MonthlyAvg = []
    for m in range(5):
          drEventWhAC_MonthlyAvg.append(batteryACOutput)

    return drEventWhAC_MonthlyAvg

performanceDLRP(3000, 2.2, 1.6, array_power_yield['yield'], invEffDefault,500,5,10,'c')

In [ ]:
# returns the annual income produced by DR performance, which is $1/kWh
#args: energy in Wh, average annual events
#returns dollar amount
def getPerformanceAmount(e, aaE=3):
    return e * 0.001 * aaE * 4

### Payback Period

In [ ]:
#args: capex, annual avoided grid costs, monthly DR revenue, annual DR performance income, annual electricity costs
def getPBP(c,g,dC,dD, p,eC):
    #get annual total economic benefit
    a = []
    for y in range(len(g)):
        a.append(g[y] + sum(dC[y]) + sum(dD[y]) + p[y])
    
    r = 0
    tI = 0
    
    #rolling total of electricity costs
    rollingElCo = 0
    
    #loop through 10 years and break the loop if the total income exceeds the CAPEX
    for y in range(1,11):
        rollingElCo += eC[y-1]
        if tI+a[y-1] >= c + rollingElCo:
            r = y-1
            break
        r = y
        tI += a[y-1]

    #determine when in the remaining year total income equals CAPEX,
    #consider that DR rates are different every month
    
    #remaining balance of CAPEX
    rtI = c + rollingElCo - tI
    
    #if our PBP is under 10 years, lets get specific
    if r < 10:
        #create a list of the total income for the remainder year by month
        mon = []
        for m in range(12):
            dr = 0
            if m >= 4 and m < 9:
                dr = dC[r][m-4] + dD[r][m-4]
            if m == 9: #add the DR performance payment at the end of the season
                dr = p[r]
            gm = g[r]/12
            mon.append(gm+dr)
        #print(mon)
        
        #print('LESS THAN OR EQUAL TO 10!')
        #print('simplified ROI: ' str(r + (rtI/a[r]))) #simplified without consider when DR occurs
        #revenue by month for remainder year
        rBM = 0
        month = 0
        for m in range(12):
            rBM += mon[m]
            if rtI <= rBM:
                month = m +1
                break
        r += month/12
                            
    #we're going to filter out ROIs over 10 years, so we just need a rough figure here to plot it later on
    #this doesn't include inflation and degradation past 10 years, nor differing monthly revenues
    else:
        #print('NOT LESS THAN 10!')
        #use max, incase the rough ROI ends up slightly less than 10
        r = max(10,r + (rtI/a[9]))

    return r

#UNCOMMENT TO TEST
##capex
#cT=1302.98635 + 6

# #annual avoided grid costs
#gT = [62.5611224869061, 63.502479696966574, 64.45800150896684, 65.42790105767229, 66.41239468488708,
#      67.41170198771056, 68.42604586751966, 69.45565257968823, 70.50075178405481, 71.56157659614948]

# #dr inc by month by year
# dT = [[15.202412956432898, 15.217837611568264, 15.174630518701857, 15.021859407269638, 14.746181494705318],
#  [15.16149258214941,  15.176799837319079,  15.134021505006283,  14.982848412258873,  14.709852237792287],
#  [15.120980546235673,  15.136171573187376,  15.093817756118627,  14.9442267175778,  14.673885565895645],
#  [15.080872782528917,  15.095948741337718,  15.054015235814862,  14.905990446334567,  14.638277867245522],
#  [15.041165265185306,  15.05612730436943,  15.014609947906358,  14.868135760085622,  14.60302556590956],
#  [15.001854008283555,  15.016703264919101,  14.975597935846036,  14.830658858457618,  14.568125121440094],
#  [14.962935065432404,  14.977672665266901,  14.936975282338325,  14.79355597877295,  14.533573028524778],
#  [14.924404529381864,  14.939031586946715,  14.89873810895291,  14.75682339567893,  14.499365816640598],
#  [14.886258531638225,  14.900776150360047,  14.860882575742178,  14.720492199543328,  14.465500049711213],
#  [14.848493242082782,  14.862902514393653,  14.82340488086241,  14.684524378331481,  14.431972325767637]]

#getPBP(cT,gT,dT)

## Main Loop

This loop generates all the possible combinations of hardware and their energy and economic characteristics based on the constraints of the battery. 

In [ ]:
#these networks are selected because they represent all possible combinations of characteristics

#myNetworks2023 = ['city hall','long island city','crown heights','randalls island', 'millwood west', 'grasslands','fresh kills','central bronx','northeast bronx']

In [ ]:

#get hourly PV yields for all possible arrays being modeled at all azimuths
yieldData = {
    '110':{},
    '180':{},
    '250':{}
}

# max allowed PV of any power station in the database
maxPV = max(hardware_df['Max PV Watts'])

for a in [110, 180, 250]:
    poa = getPOA(tilt, a)
    
    aData = {}
    for pW in range(40,maxPV+10,10):
        #get hourly power yields for a given array size (W)
        hY = getPowerYield(poa,gamma_pdc, pW)
        
        #get daily PV yields (Wh)
        dY = []
        for d in range(365):
            #print(hY['yield'].iloc[d*24:(d*24)+24].sum())
            dY.append(hY['yield'].iloc[d*24:(d*24)+24].sum())
        aData[str(pW)] = {'hourly':hY,'daily':dY}
        
    yieldData[str(a)]= aData

#display(yData['180']['360'].iloc[2000])

In [ ]:
# tempMaxYield = yieldData['180']['230']['daily']
# print((min(tempMaxYield) - dailySelfConsumptionW)* .88)
# print((max(tempMaxYield) - dailySelfConsumptionW)* .88)
# print((mean(tempMaxYield))* .89)
# print((median(tempMaxYield) - dailySelfConsumptionW)* .88)
tW = '230'
print(mean(yieldData['180'][tW]['daily']))
print(mean(yieldData['110'][tW]['daily']))
print(mean(yieldData['250'][tW]['daily']))

In [ ]:
# true = quick test, false = full run
testMode = False
testModeNetworks = testMode
testModeBatteries = testMode
testModeAzimuth = testMode
testModePV = testMode
testModeF = testMode

print('test mode: ' + str(testMode))

print('loop start time:')
print(datetime.now())

rData = {
        'network':[],
        'eventStart':[],
        'az':[],
        'pvW':[],
        'batModel':[],
        'batWh':[],
        'batWhAC':[],
        'effEff':[],#effective real efficiency
        'dischargeRateW':[],
        'capex':[],
        'dollarPerBatkWh':[],
        'dailyPVWhAC_4M':[], #year 1 PV yields AC: min, max, mean, median
        'annualPVkWhDC_degraded':[],
        'annualPVkWhAC_degraded':[],
        'avoidedGridCosts':[],
        'immediateEventAmt':[],
        'avgAutoParticipationMonthlyW_C':[],
        'avgAutoParticipationMonthlyW_D':[],
        'avgAutoParticipationAnnualW_C':[],
        'avgAutoParticipationAnnualW_D':[],
        'avgMAPw_C':[], #averaged annual auto participation W
        'avgMAPw_D':[], #averaged max auto participation W
        'totalDRratekW':[],
        'drIncMonthly_C':[],
        'drIncMonthly_D':[],
        'drIncAnnual_C':[],
        'drIncAnnual_D':[],
        'drPerformanceIncome':[],
        'annualBenefit':[],
        'LCOSS':[],
        'NPV':[],
        'PBP':[], #real rate of return includes inflation and PV degradation (does not include discount rate)
        'Cycles':[],
        'lifespan':[],
        'workingLife':[]
    }

In [ ]:
# note the 4M variable names where changed to 'Wh' and it has not been tested since. Only the variable names were changed.

if testModeNetworks:
    myNetworks = ['bowling green']
else:
    #these networks are selected because they represent all possible combinations of characteristics
    myNetworks = ['bowling green','long island city','sunnyside','crown heights','grasslands','fresh kills','williamsburg','northeast bronx','ridgewood']

#loop through all networks
for n in myNetworks:
    print(n)
    
    #loop through all batteries in database
    #comment in/out to test
    if testModeBatteries:
        bRange = range(10,25) #[20]
    else:
        bRange = range(hardware_df.shape[0])

    for b in bRange:

        #get battery info
        d = hardware_df.iloc[b]
        #print(d)

        bWh = d['Battery Wh']
        bModel = d['Model']
        
        #time in hours it takes to charge the battery
        #from 0% to 100% from the grid
        bCT = d['Full Grid Charge Hours']
        #from 80% to 100% from grid
        bFCT = d['Full Grid Charge Hours']-d['80 Per Grid Charge Hours']
        #the charge time from 20% to 100%
        practicalBCT = (d['80 Per Grid Charge Hours'] * .75) + bFCT
        
        if testModeAzimuth:
            aRange = [180]
        else:
            #loop through 3 azimuths
            aRange = [110,180,250]
            
        for a in aRange:

            if testModePV:
                pRange = [250]
            else:
                #loop through all allowed PV modules sizes, from 40 to 500 (or max PV capacity if less), incrementing by 10
                pRange = range(40,d['Max PV Watts']+10,10)
                
            for pW in pRange:
                
                #get hourly power yields for a given array size
                array_power_yield=yieldData[str(a)][str(pW)]['hourly']
                #daily power yields
                array_power_yieldDaily=yieldData[str(a)][str(pW)]['daily']
                               
                #Hardware Costs in Dollars
                capex =getCAPEX(pW,d['retail price'])
                                
                #Sum total PV generation kWDC (1st year, no PV degradation)
                pvkWhDC = array_power_yield['yield'].sum() * .001                    

                #get lists of PV WDC yield with degredation
                PVyDC = [pvkWhDC] #initialize with the year 1 undegraded generation
                for y in range(1,10):
                    PVyDC.append(getDegradedP(pvkWhDC,y))

                #convert to AC
                #assumes all PV gets utilized in battery and battery is depleted 80% at 4-hour rate
                fH = getFourHourW(bWh)

                #effective efficiency is the actualy efficiency when consuming power at the 4 hour rate
                effectiveEfficiency = getACoutput(fH,d['Inverter Watts'],invEffDefault)/fH
                
                #bat Wh with inverter losses
                bWhAC = bWh * effectiveEfficiency
                
                #subtract self-consumption from DC PV generation before converting
                annPvkWhAC = []
                for p in PVyDC:
                    annPvkWhAC.append((p - annualSelfConsumptionkWh)* effectiveEfficiency)
                #print(annPvkWhAC)
                
                minDailyYieldWhAC = (min(array_power_yieldDaily) - dailySelfConsumptionWh)* effectiveEfficiency
                maxDailyYieldWhAC = (max(array_power_yieldDaily) - dailySelfConsumptionWh)* effectiveEfficiency
                meanDailyYieldWhAC = (mean(array_power_yieldDaily) - dailySelfConsumptionWh)* effectiveEfficiency
                medianDailyYieldWhAC = (median(array_power_yieldDaily) - dailySelfConsumptionWh)* effectiveEfficiency
                
                #year 1 PV daily yields: min, max, mean, median
                #this is what needs to be consumed to avoid curtailment
                pvDailyWhAC_4M = [minDailyYieldWhAC,maxDailyYieldWhAC,meanDailyYieldWhAC,medianDailyYieldWhAC]
                
                #total annual avoided grid consumption costs based on AC solar generation
                #get lists of avoided grid costs with degredation, inflation, and self-consumption considerations
                gC_dI=[]
                for y in range(len(annPvkWhAC)):
                    gC_dI.append(utilityCosts(annPvkWhAC[y],y))

                #Automated DR participation = PV W + 25% of available battery at event start - conversion losses - self-consumption
                #get lists of CSRP DR participation rates with degredation
                dr_C = []
                for y in range(10):
                    dr_C.append(performanceCSRP(bWh,array_power_yield['yield'],invEffDefault,getStartTime(n),d['Inverter Watts'],scW, y))

                #immediate event frequency
                if testModeF:
                    fRange = [0]
                else:
                    fRange = [0,1,5]
                for i in fRange:
                    
                    rData['network'].append(n)
                    rData['eventStart'].append(getStartTime(n))
                    rData['az'].append(a)

                    #PV Watts
                    rData['pvW'].append(pW)
                    #Battery Watt Hours
                    rData['batWh'].append(bWh)
                    #battery model name
                    rData['batModel'].append(bModel)

                    rData['capex'].append(capex)
                    rData['dollarPerBatkWh'].append((capex/bWh)*.001)#this is to compare against NREL data in the discussion section
                    
                    rData['annualPVkWhDC_degraded'].append(PVyDC)

                    rData['dischargeRateW'].append(fH)
                    
                    rData['effEff'].append(effectiveEfficiency)

                    rData['batWhAC'].append(bWhAC)
                    
                    rData['dailyPVWhAC_4M'].append(pvDailyWhAC_4M)
                    
                    rData['annualPVkWhAC_degraded'].append(annPvkWhAC)

                    rData['avoidedGridCosts'].append(gC_dI)

                    rData['avgAutoParticipationMonthlyW_C'].append(dr_C)

                    #get lists of DLRP DR participation rates with degredation
                    dr_D = []
                    rData['immediateEventAmt'].append(i)
                    for y in range(10):
                        #determine if its a contingency year or an emergency year
                        if i == 0:
                            eventType = 'c'
                        elif i == 1:
                            if y==0:
                                eventType ='i'
                            else:
                                eventType = 'c'
                        elif i == 5:
                            if y<5:
                                eventType ='i'
                            else:
                                eventType = 'c'
                                
                        dr_D.append(performanceDLRP(bWh,practicalBCT, bFCT, array_power_yield['yield'],invEffDefault,d['Inverter Watts'],scW, y,eventType))
                    rData['avgAutoParticipationMonthlyW_D'].append(dr_D)

                    mAP_C = []
                    for y in range(10):
                        mAP_C.append(sum(dr_C[y])/len(dr_D[y]))
                    rData['avgAutoParticipationAnnualW_C'].append(mAP_C)

                    mAP_D = []
                    for y in range(10):
                        mAP_D.append(sum(dr_D[y])/len(dr_D[y]))
                    rData['avgAutoParticipationAnnualW_D'].append(mAP_D)

                    rData['avgMAPw_C'].append(sum(mAP_C)/len(mAP_C))
                    rData['avgMAPw_D'].append(sum(mAP_D)/len(mAP_D))


                    #monthly income from DR participation
                    #the DR rate for each program
                    dDRrkW= getDLRPrate(n)
                    cDRrkW= getCSRPrate(n)

                    tDRrkW = dDRrkW + cDRrkW
                    rData['totalDRratekW'].append(tDRrkW)

                    #monthly income from DR
                    drInc_C = []
                    drInc_D = []
                    for y in range(0,10):
                        drInc_C.append([m * 0.001 * cDRrkW for m in dr_C[y]])
                        drInc_D.append([m * 0.001 * dDRrkW for m in dr_D[y]])
                    rData['drIncMonthly_C'].append(drInc_C)
                    rData['drIncMonthly_D'].append(drInc_D)

                    #annual income from DR participation
                    drAnnInc_C = []
                    for y in drInc_C:
                        drAnnInc_C.append(sum(y))
                    rData['drIncAnnual_C'].append(drAnnInc_C)

                    drAnnInc_D = []
                    for y in drInc_D:
                        drAnnInc_D.append(sum(y))
                    rData['drIncAnnual_D'].append(drAnnInc_D)

                    #this isn't factored in to monthly income.
                    #for the  purposes of this model, it is paid out after the DR season is over. i.e. with September payments
                    drPer = []
                    for y in range(10):
                        #assumes 3 CSRP events per year and 0 DLRP events per year
                        drPer.append(getPerformanceAmount(mAP_C[y]))
                    rData['drPerformanceIncome'].append(drPer)

                    annInc = []
                    for y in range(10):
                        #add avoided  grid costs, reservation income from CSRP and DLRP, and CSRP performance income
                        annInc.append(gC_dI[y] + drAnnInc_C[y] + drAnnInc_D[y] + drPer[y])
                    rData['annualBenefit'].append(annInc)
                    
                    #assumes 95% efficient grid charging
                    annElecCosts = []
                    for y in range(10):
                        #assume 4 events per year, this includes 3 CSRP real events and 1 DLRP test event
                        annElecCosts.append(utilityCosts((bWh * 0.001 * LiFePOdod * 4) / .95,y))
                    
                    #real(simple) PBP is varies by year and month, but is not discounted
                    #args: capex, annual avoided grid costs, monthly DR revenue,
                    #annual DR performance income, annual electricity costs
                    rPBP = getPBP(capex, gC_dI,drInc_C, drInc_D,drPer,annElecCosts)
                    rData['PBP'].append(rPBP)

                    rData['Cycles'].append(d['Cycles'])
                    #life span assumes 1 full cycle a day
                    lS = d['Cycles']/365
                    rData['lifespan'].append(lS)
                    #working life caps the lifespan at 10 years
                    wL = min(10,lS)
                    rData['workingLife'].append(wL)

                    #args: working life, capex, annual avoided grid costs, monthly DR revenue,
                    #annual DR performance income, annual electricity costs
                    #print([wL, capex, gC_dI,drInc_C, drInc_D,drPer,annElecCosts])
                    rNPV = getNPV(wL, capex, gC_dI,drInc_C, drInc_D,drPer,annElecCosts)
                    rData['NPV'].append(rNPV)
                    
                    #pv W, battery Wh, capex, working years, annual PV yield kWAC (list), effective efficiency of inverter
                    rData['LCOSS'].append(getLCOSS(pW,bWh,capex,wL,annPvkWhAC,effectiveEfficiency))

print('loop end time:')
print(datetime.now())

    #load data into a DataFrame object:
results_df = pd.DataFrame(rData)
    #nData[n]=productTest_df
print(str(results_df.shape[0])+ ' total hardware combinations generated')

ts = datetime.now().timestamp()
results_df.to_csv('results/results_df_' + str(math.floor(ts)) +'.csv')
print('file saved as: ' + 'results/results_df_' + str(math.floor(ts)) +'.csv')


In [ ]:
results_df.keys()

In [ ]:
# results_df['minDRkWperf']/results_df['realRoi'].head()
# results_df['minDRWperfH'] = (results_df['minDRkWperf']/results_df['realRoi']/5) * 1000

In [ ]:
results_df.describe()

In [ ]:
# Make it 14x7 inch
plt.figure(figsize=(14,7)) # Make it 14x7 inch
#plt.style.use('seaborn-whitegrid') # nice and clean grid
plt.hist(results_df[results_df['PBP']<20]['PBP'], bins=200, facecolor = '#2ab0ff', edgecolor='#169acf', linewidth=0.5)
plt.title('Hardware Payback Period Histogram (Raw Results)') 
plt.xlabel('Payback Years') 
plt.ylabel('Number of Hardware Combinations') 
plt.show()

In [ ]:
results_df[results_df['NPV'] >= 0]
#results_df['NPV'] 

# Make it 14x7 inch
plt.figure(figsize=(14,7)) # Make it 14x7 inch
#plt.style.use('seaborn-whitegrid') # nice and clean grid
plt.hist(results_df['NPV'], bins=200, facecolor = '#2ab0ff', edgecolor='#169acf', linewidth=0.5)
plt.title('Hardware NPV Histogram (Raw Results)') 
plt.xlabel('Value') 
plt.ylabel('Number of Hardware Combinations') 
plt.show()

## Filtering - PBP

In [ ]:
#filter out anything with an PBP over its working years
favPayback_df = results_df[results_df['PBP'] <= results_df['workingLife']] 

favPayback_df_Z = favPayback_df[favPayback_df['immediateEventAmt'] == 0] 
favPayback_df_O = favPayback_df[favPayback_df['immediateEventAmt'] == 1] 
favPayback_df_F = favPayback_df[favPayback_df['immediateEventAmt'] == 5] 

print('Total Combinations')
print(str(favPayback_df.shape[0])+ ' total favorable PBP hardware combinations generated')
print(str(favPayback_df_Z.shape[0])+ ' total favorable PBP hardware combinations generated - Zero Immediate Event')
print(str(favPayback_df_O.shape[0])+ ' total favorable PBP hardware combinations generated - One Immediate Event')
print(str(favPayback_df_F.shape[0])+ ' total favorable PBP hardware combinations generated - Five Immediate Event')

print('Eastern')
print(str(favPayback_df[favPayback_df['az']==110].shape[0])+ ' eastern favorable PBP hardware combinations generated')
print(str(favPayback_df_Z[favPayback_df_Z['az']==110].shape[0])+ ' eastern favorable PBP hardware combinations generated - Zero Immediate Event')
print(str(favPayback_df_O[favPayback_df_O['az']==110].shape[0])+ ' eastern favorable PBP hardware combinations generated - One Immediate Event')
print(str(favPayback_df_F[favPayback_df_F['az']==110].shape[0])+ ' eastern favorable PBP hardware combinations generated - Five Immediate Event')

print('Southern')
print(str(favPayback_df[favPayback_df['az']==180].shape[0])+ ' southern favorable PBP hardware combinations generated')
print(str(favPayback_df_Z[favPayback_df_Z['az']==180].shape[0])+ ' southern favorable PBP hardware combinations generated - Zero Immediate Event')
print(str(favPayback_df_O[favPayback_df_O['az']==180].shape[0])+ ' southern favorable PBP hardware combinations generated - One Immediate Event')
print(str(favPayback_df_F[favPayback_df_F['az']==180].shape[0])+ ' southern favorable PBP hardware combinations generated - Five Immediate Event')

print('Western')
print(str(favPayback_df[favPayback_df['az']==250].shape[0])+ ' westerm favorable PBP hardware combinations generated')
print(str(favPayback_df_Z[favPayback_df_Z['az']==250].shape[0])+ ' westerm favorable PBP hardware combinations generated - Zero Immediate Event')
print(str(favPayback_df_O[favPayback_df_O['az']==250].shape[0])+ ' westerm favorable PBP hardware combinations generated - One Immediate Event')
print(str(favPayback_df_F[favPayback_df_F['az']==250].shape[0])+ ' westerm favorable PBP hardware combinations generated - Five Immediate Event')

## Filtering - PBP

In [ ]:
results_df['PBP'].describe()

In [ ]:
favNPV_df = results_df[results_df['PBP']>=0]
favNPV_df.describe()

In [ ]:
results_df['NPV'].describe()

In [ ]:
favNPV_df = results_df[results_df['NPV']>=0]
favNPV_df.describe()